In [23]:
# import required packages
# from fastai2.tabular.all import *
import xgboost as xgb
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from helpers import *

pd.options.display.max_columns = None

In [24]:
# path to project directory
path = Path('./')

In [25]:
# read in training dataset
train_df = pd.read_csv(path/'data/train_v7.csv', index_col=0, dtype={'season':str,
                                                                     'squad':str,
                                                                     'comp':str})

In [26]:
train_df.shape

(109893, 58)

In [27]:
# features for model
season = '2021'
cat_vars = ['season', 'position', 'was_home']
cont_vars = ['gw', 'minutes']
dep_var = ['total_points']

# find the latest gameweek
last_gw = train_df['gw'][train_df['season'] == season].max()

if np.isnan(last_gw): 
    next_gw = 1 
else: 
    next_gw = last_gw + 1
    
next_gw

32

In [28]:
# read in remaining_season.csv and sort
remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', 
                                  index_col=0).sort_values(by=['gw', 'team'])

# update fields needed for processing
remaining_season_df['season'] = season
remaining_season_df['total_points'] = 0

# need to add an ordered date-like field  
remaining_season_df['kickoff_time'] = [str(9000 + x) + '-99-99' for x in remaining_season_df['match_no']]

# remove match_no, don't need it anymore
remaining_season_df = remaining_season_df.drop('match_no', axis=1)

# # append remaining season to end of training set
train_df = train_df.append(remaining_season_df, sort=False).reset_index(drop=True)

In [29]:
# # read in remaining_season.csv and sort
# remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', 
#                                   index_col=0).sort_values(by=['gw', 'team'])

# remaining_season_df['season'] = season
# remaining_season_df['total_points'] = 0
# remaining_season_df['kickoff_time'] = '9999-99-99'

# # append remaining season to end of training set
# train_df = train_df.append(remaining_season_df, sort=False).reset_index(drop=True)

In [30]:
# add player lag features
lag_train_df, team_lag_vars = team_lag_features(train_df, ['total_points'], ['all', 3, 5, 10, 20])
lag_train_df, player_lag_vars = player_lag_features(lag_train_df, ['total_points'], ['all', 1, 2, 3, 5, 10, 20])

# add team conceded fields for opposition
pattern = re.compile('total_points_team_pg_last_.*_opponent')
team_lag_vars += [x.replace('team', 'team_conceded') for x in team_lag_vars if pattern.match(x)]

# but remove scored for opposition
pattern = re.compile('total_points_team_pg_last_.*_opponent')
team_lag_vars = [x for x in team_lag_vars if not pattern.match(x)]

In [31]:
team_lag_vars

['total_points_team_pg_last_all',
 'total_points_team_pg_last_3',
 'total_points_team_pg_last_5',
 'total_points_team_pg_last_10',
 'total_points_team_pg_last_20',
 'total_points_team_conceded_pg_last_all_opponent',
 'total_points_team_conceded_pg_last_3_opponent',
 'total_points_team_conceded_pg_last_5_opponent',
 'total_points_team_conceded_pg_last_10_opponent',
 'total_points_team_conceded_pg_last_20_opponent']

In [32]:
# we want to set gw and season as ordered categorical variables
# need lists with ordered categories
ordered_gws = list(range(1,39))
ordered_seasons = ['1617', '1718', '1819', '1920', '2021']

# set as categories with correct order 
lag_train_df['gw'] = lag_train_df['gw'].astype('category')
lag_train_df['season'] = lag_train_df['season'].astype('category')

lag_train_df['gw'].cat.set_categories(ordered_gws, ordered=True, inplace=True)
lag_train_df['season'].cat.set_categories(ordered_seasons, ordered=True, inplace=True)

In [33]:
lag_train_df[lag_train_df['player'] == 'Bernd Leno'].tail(25)

player  gw  position  minutes     team             opponent_team  \
97918   Bernd Leno  14         1     90.0  Arsenal                   Everton   
98530   Bernd Leno  15         1     90.0  Arsenal                   Chelsea   
99123   Bernd Leno  16         1     90.0  Arsenal  Brighton and Hove Albion   
99622   Bernd Leno  17         1     90.0  Arsenal      West Bromwich Albion   
100175  Bernd Leno  18         1     90.0  Arsenal            Crystal Palace   
100634  Bernd Leno  19         1     90.0  Arsenal          Newcastle United   
101553  Bernd Leno  20         1     90.0  Arsenal               Southampton   
102198  Bernd Leno  21         1     90.0  Arsenal         Manchester United   
102849  Bernd Leno  22         1     71.0  Arsenal   Wolverhampton Wanderers   
103512  Bernd Leno  23         1      0.0  Arsenal               Aston Villa   
104205  Bernd Leno  24         1     90.0  Arsenal                     Leeds   
104987  Bernd Leno  25         1     90.0  Arsenal           Manchester City   
105779  Bernd Leno  26         1     90.0  Arsenal            Leicester City   
106886  Bernd Leno  27         1     90.0  Arsenal                   Burnley   
107633  Bernd Leno  28         1     90.0  Arsenal         Tottenham Hotspur   
108272  Bernd Leno  29         1     90.0  Arsenal           West Ham United   
108596  Bernd Leno  30         1     90.0  Arsenal                 Liverpool   
109283  Bernd Leno  31         1     90.0  Arsenal          Sheffield United   
109900  Bernd Leno  32         1     90.0  Arsenal                    Fulham   
110590  Bernd Leno  33         1     90.0  Arsenal                   Everton   
111143  Bernd Leno  34         1     90.0  Arsenal          Newcastle United   
111834  Bernd Leno  35         1     90.0  Arsenal      West Bromwich Albion   
112525  Bernd Leno  36         1     90.0  Arsenal                   Chelsea   
113216  Bernd Leno  37         1     90.0  Arsenal            Crystal Palace   
113907  Bernd Leno  38         1     90.0  Arsenal  Brighton and Hove Albion   

        relative_market_value_team  relative_market_value_opponent_team  \
97918                     1.317380                             1.145049   
98530                     1.317380                             1.859440   
99123                     1.316625                             0.541862   
99622                     1.261632                             0.258537   
100175                    1.264284                             0.475724   
100634                    1.272515                             0.604033   
101553                    1.343890                             0.538945   
102198                    1.353529                             1.679166   
102849                    1.253765                             0.969065   
103512                    1.253765                             0.749770   
104205                    1.252915                             0.439832   
104987                    1.252915                             2.397273   
105779                    1.252915                             1.059550   
106886                    1.252936                             0.342974   
107633                    1.279916                             1.586900   
108272                    1.280364                             0.652307   
108596                    1.280364                             2.355067   
109283                    1.280737                             0.356395   
109900                    1.280737                             0.520365   
110590                    1.280737                             1.078749   
111143                    1.280737                             0.589405   
111834                    1.280737                             0.282807   
112525                    1.280737                             1.818829   
113216                    1.280737                             0.474183   
113907                    1.280737                      

In [34]:
# create dataset with latest lag numbers for all future gameweeks
valid_season = '2021'
valid_gw = next_gw
valid_len = 38 - next_gw + 1

train_predict_df, train_idx, valid_idx = create_lag_train(lag_train_df, 
                                                      cat_vars, cont_vars, 
                                                      player_lag_vars, team_lag_vars, dep_var,
                                                      valid_season, valid_gw, valid_len)

In [35]:
team_lag_vars

['total_points_team_pg_last_all',
 'total_points_team_pg_last_3',
 'total_points_team_pg_last_5',
 'total_points_team_pg_last_10',
 'total_points_team_pg_last_20',
 'total_points_team_conceded_pg_last_all_opponent',
 'total_points_team_conceded_pg_last_3_opponent',
 'total_points_team_conceded_pg_last_5_opponent',
 'total_points_team_conceded_pg_last_10_opponent',
 'total_points_team_conceded_pg_last_20_opponent']

In [36]:
# take a look at the dataframe
train_predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114733 entries, 0 to 114732
Data columns (total 33 columns):
 #   Column                                           Non-Null Count   Dtype   
---  ------                                           --------------   -----   
 0   gw                                               114733 non-null  category
 1   minutes                                          114733 non-null  float64 
 2   minutes_last_1                                   114733 non-null  float64 
 3   minutes_last_10                                  114733 non-null  float64 
 4   minutes_last_2                                   114733 non-null  float64 
 5   minutes_last_20                                  114733 non-null  float64 
 6   minutes_last_3                                   114733 non-null  float64 
 7   minutes_last_5                                   114733 non-null  float64 
 8   minutes_last_all                                 114733 non-null  float64 
 9   oppo

In [37]:
# split out dependent variable
X, y = train_predict_df[cat_vars + cont_vars + player_lag_vars + team_lag_vars].copy(), train_predict_df[dep_var].copy()

In [38]:
# since position is categorical, it should be a string
X['position'] = X['position'].apply(str)

# need to transform season
enc = LabelEncoder()
X['season'] = enc.fit_transform(X['season'])
X_dict = X.to_dict("records")

# Create the DictVectorizer object: dv
dv = DictVectorizer(sparse=False, separator='_')

# Apply dv on df: df_encoded
X_encoded = dv.fit_transform(X_dict)

X_df = pd.DataFrame(X_encoded, columns=dv.feature_names_)

In [39]:
# split out training and prediction sets
X_train = X_df.iloc[train_idx]
y_train = y.iloc[train_idx]
X_test = X_df.iloc[valid_idx]
y_test = y.iloc[valid_idx]

In [40]:
# instatiate and train XGB Regressor
model_params = {"gamma": 0.42,
                "learning_rate": 0.047, 
                "max_depth": 4,
                "n_estimators": 171,
                "subsample": 0.6}

xg_reg = xgb.XGBRegressor(objective="reg:squarederror",
                          gamma=model_params['gamma'], 
                          learning_rate=model_params['learning_rate'], 
                          max_depth=model_params['max_depth'],
                          n_estimators=model_params['n_estimators'],
                          subsample=model_params['subsample'])

xg_reg.fit(X_train, y_train)
preds = xg_reg.predict(X_test)

In [41]:
preds

array([ 0.011172  ,  0.02605698, -0.02088863, ...,  0.00367564,
        2.7390924 ,  0.00367564], dtype=float32)

In [42]:
remaining_season_df['predicted_points'] = preds

In [43]:
remaining_season_df.head(50)

gw         team    opponent_team                            player  \
33  32      Arsenal           Fulham                        Mesut Özil   
34  32      Arsenal           Fulham         Sokratis Papastathopoulos   
35  32      Arsenal           Fulham        David Luiz Moreira Marinho   
36  32      Arsenal           Fulham         Pierre-Emerick Aubameyang   
37  32      Arsenal           Fulham                     Cédric Soares   
38  32      Arsenal           Fulham               Alexandre Lacazette   
39  32      Arsenal           Fulham                  Shkodran Mustafi   
40  32      Arsenal           Fulham                        Bernd Leno   
41  32      Arsenal           Fulham                      Granit Xhaka   
42  32      Arsenal           Fulham                        Pablo Marí   
43  32      Arsenal           Fulham                   Héctor Bellerín   
44  32      Arsenal           Fulham                    Calum Chambers   
45  32      Arsenal           Fulham                    Sead Kolasinac   
46  32      Arsenal           Fulham                       Rob Holding   
47  32      Arsenal           Fulham                    Kieran Tierney   
48  32      Arsenal           Fulham                      Nicolas Pépé   
49  32      Arsenal           Fulham                    Lucas Torreira   
50  32      Arsenal           Fulham                      Reiss Nelson   
51  32      Arsenal           Fulham                    Edward Nketiah   
52  32      Arsenal           Fulham                  Emile Smith Rowe   
53  32      Arsenal           Fulham                       Bukayo Saka   
54  32      Arsenal           Fulham                  Matteo Guendouzi   
55  32      Arsenal           Fulham  Gabriel Teodoro Martinelli Silva   
56  32      Arsenal           Fulham                    William Saliba   
57  32      Arsenal           Fulham                       Mathew Ryan   
58  32      Arsenal           Fulham           Willian Borges Da Silva   
59  32      Arsenal           Fulham                 Gabriel Magalhães   
60  32      Arsenal           Fulham         Daniel Ceballos Fernández   
61  32      Arsenal           Fulham     Mohamed Naser El Sayed Elneny   
62  32      Arsenal           Fulham                        Matt Macey   
63  32      Arsenal           Fulham              Rúnar Alex Rúnarsson   
64  32      Arsenal           Fulham                     Thomas Partey   
65  32      Arsenal           Fulham                   Martin Ødegaard   
66  32      Arsenal           Fulham                      Miguel Azeez   
67  32  Aston Villa  Manchester City                 Emiliano Martínez   
68  32  Aston Villa  Manchester City                        Tom Heaton   
69  32  Aston Villa  Manchester City                 Ahmed El Mohamady   
70  32  Aston Villa  Manchester City                    Henri Lansbury   
71  32  Aston Villa  Manchester City                       Neil Taylor   
72  32  Aston Villa  Manchester City                         Jed Steer   
73  32  Aston Villa  Manchester City                   Conor Hourihane   
74  32  Aston Villa  Manchester City    José Ignacio Peleteiro Romallo   
75  32  Aston Villa  Manchester City                      Ørjan Nyland   
76  32  Aston Villa  Manchester City                      Bjorn Engels   
77  32  Aston Villa  Manchester City                     Jack Grealish   
78  32  Aston Villa  Manchester City                       John McGinn   
79  32  Aston Villa  Manchester City                     Kortney Hause   
80  32  Aston Villa  Manchester City      Mahmoud Ahmed Ibrahim Hassan   
81  32  Aston Villa  Manchester City                      Tyrone Mings   
82  32  Aston Villa  Manchester City                      Matt Targett   

    position  price  play_proba   short_name  was_home  \
33         3    6.7        0.00         Özil      True   
34         2    4.8        0.00     Sokratis      True   
35         2    5.4        0.00   David Luiz      True 

In [44]:
remaining_season_df[remaining_season_df['gw'] == next_gw].sort_values(by='predicted_points', ascending=False).head(50)

gw                      team             opponent_team  \
6     32         Tottenham Hotspur               Southampton   
246   32         Manchester United                   Burnley   
2572  32         Tottenham Hotspur                   Everton   
2612  32                 Liverpool                     Leeds   
216   32            Leicester City      West Bromwich Albion   
38    32                   Arsenal                    Fulham   
241   32         Manchester United                   Burnley   
2508  32           Manchester City               Aston Villa   
10    32         Tottenham Hotspur               Southampton   
208   32            Leicester City      West Bromwich Albion   
126   32                   Chelsea  Brighton and Hove Albion   
8     32         Tottenham Hotspur               Southampton   
103   32                   Chelsea  Brighton and Hove Albion   
245   32         Manchester United                   Burnley   
40    32                   Arsenal                    Fulham   
2613  32                 Liverpool                     Leeds   
249   32         Manchester United                   Burnley   
332   32   Wolverhampton Wanderers          Sheffield United   
2576  32         Tottenham Hotspur                   Everton   
250   32         Manchester United                   Burnley   
106   32                   Chelsea  Brighton and Hove Albion   
1     32         Tottenham Hotspur               Southampton   
308   32   Wolverhampton Wanderers          Sheffield United   
2574  32         Tottenham Hotspur                   Everton   
257   32         Manchester United                   Burnley   
2504  32           Manchester City               Aston Villa   
125   32                   Chelsea  Brighton and Hove Albion   
2519  32           Manchester City               Aston Villa   
2657  32      West Bromwich Albion            Leicester City   
2616  32                 Liverpool                     Leeds   
27    32         Tottenham Hotspur               Southampton   
243   32         Manchester United                   Burnley   
313   32   Wolverhampton Wanderers          Sheffield United   
2700  32           West Ham United          Newcastle United   
2666  32      West Bromwich Albion            Leicester City   
2539  32  Brighton and Hove Albion                   Chelsea   
317   32   Wolverhampton Wanderers          Sheffield United   
146   32                   Everton         Tottenham Hotspur   
2610  32                 Liverpool                     Leeds   
2622  32                 Liverpool                     Leeds   
64    32                   Arsenal                    Fulham   
152   32                   Everton         Tottenham Hotspur   
2567  32         Tottenham Hotspur                   Everton   
201   32            Leicester City      West Bromwich Albion   
311   32   Wolverhampton Wanderers          Sheffield United   
212   32            Leicester City      West Bromwich Albion   
252   32         Manchester United                   Burnley   
219   32            Leicester City      West Bromwich Albion   
2679  32                   Burnley         Manchester United   
224   32            Leicester City      West Bromwich Albion   

                              player  position  price  play_proba  \
6                         Harry Kane         4   11.7         1.0   
246    Bruno Miguel Borges Fernandes         3   11.6         1.0   
2572                      Harry Kane         4   11.7         1.0   
2612                   Mohamed Salah         3   12.5         1.0   
216                Kelechi Iheanacho         4    6.0         1.0   
38               Alexandre Lacazette         4    8.3         1.0   
241                       Paul Pogba         3    7.6         1.0   
2508       Gabriel Fernando de Jesus         4    9.2         1.0   
10    Lucas Rodrigues Moura da Silva         3    6.6         1.0   
208                      Jamie Vardy         4   10

In [45]:
# rename previous week's predictions file
# generate previous week's filename
last_gw = next_gw - 1
filename = 'history/2020-21/predictions_gw' + str(last_gw) + '.csv'
# windows
! move predictions.csv $filename
# linux
# ! mv predictions.csv $filename

        1 file(s) moved.


In [46]:
# write to predictions.csv
## RENAME PREVIOUS WEEK FIRST
predictions = remaining_season_df[['gw', 'player', 'team', 'opponent_team', 
                                   'position', 'price', 'play_proba', 'short_name',
                                   'was_home', 'predicted_points', 'minutes']]
predictions.to_csv('predictions.csv')